Имеются данные adult.csv 
Целевой переменной является уровень дохода income 
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv( 'adult.csv' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
data.info() # смотрим качетсво данных, пропущенные значения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
# выбираем признаки
selectedcolumns = data[ [ 'educational-num', 'occupation', 'hours-per-week', 'income' ] ]
selectedcolumns

,educational-num,occupation,hours-per-week,income
0,7,Machine-op-inspct,40,<=50K
1,9,Farming-fishing,50,<=50K
2,12,Protective-serv,40,>50K
3,10,Machine-op-inspct,40,>50K
4,10,?,30,<=50K
...,...,...,...,...
48837,12,Tech-support,38,<=50K
48838,9,Machine-op-inspct,40,>50K
48839,9,Adm-clerical,40,<=50K
48840,9,Adm-clerical,20,<=50K


In [5]:
selectedcolumns[ selectedcolumns['occupation'] == '?' ].head()

,educational-num,occupation,hours-per-week,income
4,10,?,30,<=50K
6,9,?,40,<=50K
13,9,?,35,<=50K
22,4,?,6,<=50K
35,9,?,40,<=50K


In [6]:
len(selectedcolumns[ selectedcolumns['occupation'] == '?' ])

2809

In [7]:
# столбец occupation является категориальной переменной, переведем в значения 0 и 1
x = pd.get_dummies( selectedcolumns, columns = [ 'occupation' ] )

# столбец income является целевой переменной, occupation_? - значение столбца occupation с неизвестным значением,
# удалим их. 
x.drop(['income', 'occupation_?'], axis='columns', inplace=True)
x.head()

,educational-num,hours-per-week,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,7,40,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,9,50,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,12,40,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,10,40,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,10,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# целевой столбец income является категориальной переменной (>50K и <=50K), переведем ее значения в числа

In [9]:
le = LabelEncoder()
le.fit( data['income'] )

LabelEncoder()

In [10]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [11]:
# записываем в переменную y преобразованный столбец income

y = pd.Series( data = le.transform( data['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


model = make_pipeline(StandardScaler(),LogisticRegression(max_iter=500))

In [13]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [14]:
model.fit( x_train, y_train )
predictions = model.predict( x_test )
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
# l = list(zip( predictions, y))
# k = 0
# p = 0
# for i in range(len(l)):
#     if l[i][0] == l[i][1]:
#         k += 1
#     else:
#         p += 1
# print( k/(k+p)*100, p/(k+p)*100)
        

69.37145976933051 30.628540230669486


In [16]:
model.score(x_train, y_train)

0.7857498025680775

In [17]:
model.score(x_test, y_test)

0.7942400873541254

Точность получилась так себе

In [18]:
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC())
clf.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [19]:
clf.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0])

In [20]:
clf.score(x_train, y_train)

0.7872415104273305

In [21]:
clf.score(x_test, y_test)

0.7972428854159558

Немного лучше, но совсем немного